# <span style="color:darkblue"> Lecture 15 - Merging Data </span>

<font size = "5">

In the previous class we covered ...

- Aggregate Statistics
- Merge aggregate stats

In this class we will cover ...

- More database merging!
- Emphasize importance of cleaning before merging
- Database concatenation

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<font size = "5">

Read dataset on car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [3]:
results_raw  = pd.read_csv("data_raw/results.csv")
races_raw    = pd.read_csv("data_raw/races.csv")
circuits_raw = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

Multi-file datasets can be visualized with an ...

- "Entity Relationship Diagram" (ERD)
- How the identifiers in each table are connected
- Complement to the "codebook"

<img src="figures/erd_f1_simple.png" alt="drawing" width="600"/>


<font size = "5">

Start by opening datasets!

- Check columns with similar names

In [4]:
# We extract all the unique values in races_raw["name"] and circuits_raw["name"]
# We use "sort_values()" to make it easier to compare the variables
# The "codebook/f1_codebook.pdf" file shows that the content is indeed different

unique_data_races    = pd.unique(races_raw["name"].sort_values())
unique_data_circuits = pd.unique(circuits_raw["name"].sort_values())


# <span style="color:darkblue"> II. Dictionaries + Renaming </span>

<font size = "5">

A dictionary is another way to store data. 

- Defined with curly brackets "{...}"
- Different fields are separated by a comma
- Assign values to a field with a colon ":"

<font size = "5">

Dictionaries + Pandas

In [5]:
# This is an example of a pandas data frame created from a dictionary
# This example illustrates the basic syntax of a dictionary

car_dictionary = {"car_model": ["Ferrari","Tesla","BMW"],
                  "year":      ["2018","2023","2022"] }

pd.DataFrame(car_dictionary)


,car_model,year
0,Ferrari,2018
1,Tesla,2023
2,BMW,2022


<font size = "5">

Rename columns with dictionaries

``` {"old_name": "new_name"} ```

In [19]:
# We first define the dictionary
# Change the pipe ".rename(...)" to rename the columns
# Dictionaries can flexibly accommodate single values or list after ":"

dict_rename_circuits = {"name": "circuit_name"}
circuits = circuits_raw.rename(columns = dict_rename_circuits)
print(circuits)

    circuitId   circuitRef                          circuit_name  \
0           1  albert_park        Albert Park Grand Prix Circuit   
1           2       sepang          Sepang International Circuit   
2           3      bahrain         Bahrain International Circuit   
3           4    catalunya        Circuit de Barcelona-Catalunya   
4           5     istanbul                         Istanbul Park   
..        ...          ...                                   ...   
72         75     portimao    Autódromo Internacional do Algarve   
73         76      mugello  Autodromo Internazionale del Mugello   
74         77       jeddah               Jeddah Corniche Circuit   
75         78       losail          Losail International Circuit   
76         79        miami         Miami International Autodrome   

        location       country       lat        lng  alt  \
0      Melbourne     Australia -37.84970  144.96800   10   
1   Kuala Lumpur      Malaysia   2.76083  101.73800   18   
2  

<font size = "5">
Check that ".rename()" worked

In [7]:
# Extract the column names of the "raw" and "clean" data

print("Old List:")
print(circuits_raw.columns.values)
print("")
print("New List:")
print(circuits.columns.values)


Old List:
['circuitId' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt'
 'url']

New List:
['circuitId' 'circuitRef' 'circuit_name' 'location' 'country' 'lat' 'lng'
 'alt' 'url']


<font size = 5>

Try it yourself!

- Create a dictionary to rename "name" to "race_name"
- Rename this column in the "races_raw" dataset
- Store the output in a new dataset called "races"

In [20]:
# Write your own code
dict_rename_races = {"name": "race_name"}
races = races_raw.rename(columns = dict_rename_races)
print(races)

      raceId  year  round  circuitId                 race_name        date  \
0          1  2009      1          1     Australian Grand Prix  2009-03-29   
1          2  2009      2          2      Malaysian Grand Prix  2009-04-05   
2          3  2009      3         17        Chinese Grand Prix  2009-04-19   
3          4  2009      4          3        Bahrain Grand Prix  2009-04-26   
4          5  2009      5          4        Spanish Grand Prix  2009-05-10   
...      ...   ...    ...        ...                       ...         ...   
1097    1116  2023     19         69  United States Grand Prix  2023-10-22   
1098    1117  2023     20         32    Mexico City Grand Prix  2023-10-29   
1099    1118  2023     21         18      São Paulo Grand Prix  2023-11-05   
1100    1119  2023     22         80      Las Vegas Grand Prix  2023-11-19   
1101    1120  2023     23         24      Abu Dhabi Grand Prix  2023-11-26   

          time                                                u

# <span style="color:darkblue"> II. Merging </span>


<font size = "5">

Extracting specific columns from dataset

In [9]:
circuits[["circuitId","circuit_name"]]

,circuitId,circuit_name
0,1,Albert Park Grand Prix Circuit
1,2,Sepang International Circuit
2,3,Bahrain International Circuit
3,4,Circuit de Barcelona-Catalunya
4,5,Istanbul Park
...,...,...
72,75,Autódromo Internacional do Algarve
73,76,Autodromo Internazionale del Mugello
74,77,Jeddah Corniche Circuit
75,78,Losail International Circuit


<font size = "5">

Merge datasets

<img src="figures/merge_goal.png" alt="drawing" width="500"/>


```pd.merge(data1,data2,on,how)```

- Strive to merge only specific columns of data2
- Avoid merging all columns
- Keeping it simple gives you more control over the output

In [22]:
# The "pd.merge()" command combines the information from both datasets
# The first argument is the "primary" datasets
# The second argument is the "secondary" dataset (much include the "on" column)
# The "on" is the common variable that is used for merging
# how = "left" tells Python that the left dataset is the primary one

races_merge = pd.merge(races_raw,
                       circuits[["circuitId","circuit_name"]],
                       on = "circuitId",
                       how = "left")
print(races_merge.head())

   raceId  year  round  circuitId                   name        date  \
0       1  2009      1          1  Australian Grand Prix  2009-03-29   
1       2  2009      2          2   Malaysian Grand Prix  2009-04-05   
2       3  2009      3         17     Chinese Grand Prix  2009-04-19   
3       4  2009      4          3     Bahrain Grand Prix  2009-04-26   
4       5  2009      5          4     Spanish Grand Prix  2009-05-10   

       time                                                url fp1_date  \
0  06:00:00  http://en.wikipedia.org/wiki/2009_Australian_G...       \N   
1  09:00:00  http://en.wikipedia.org/wiki/2009_Malaysian_Gr...       \N   
2  07:00:00  http://en.wikipedia.org/wiki/2009_Chinese_Gran...       \N   
3  12:00:00  http://en.wikipedia.org/wiki/2009_Bahrain_Gran...       \N   
4  12:00:00  http://en.wikipedia.org/wiki/2009_Spanish_Gran...       \N   

  fp1_time fp2_date fp2_time fp3_date fp3_time quali_date quali_time  \
0       \N       \N       \N       \N       

In [23]:
# Another example of merging

results_merge = pd.merge(results_raw,
                         races_raw[["raceId","date"]],
                         on = "raceId",
                         how = "left")
print(results_merge.head())

   resultId  raceId  driverId  constructorId number  grid position  \
0         1      18         1              1     22     1        1   
1         2      18         2              2      3     5        2   
2         3      18         3              3      7     7        3   
3         4      18         4              4      5    11        4   
4         5      18         5              1     23     3        5   

  positionText  positionOrder  points  laps         time milliseconds  \
0            1              1    10.0    58  1:34:50.616      5690616   
1            2              2     8.0    58       +5.478      5696094   
2            3              3     6.0    58       +8.163      5698779   
3            4              4     5.0    58      +17.181      5707797   
4            5              5     4.0    58      +18.014      5708630   

  fastestLap rank fastestLapTime fastestLapSpeed  statusId        date  
0         39    2       1:27.452         218.300         1  2008-03

<font size = "5">
<span style="color:red"> Common pitfall: </span> What happens if you don't rename?

In [24]:
# The following code merges the raw data
# which has the "name" column in "races_raw" and "circuits_raw"

races_merge_pitfall = pd.merge(races_raw,
                               circuits_raw[["circuitId","name"]],
                               on = "circuitId",
                               how = "left")

# Python will internally rename the columns "name_x" (for the left dataset)
# and "name_y" (for the right dataset)

print(races_merge_pitfall.columns.values)


['raceId' 'year' 'round' 'circuitId' 'name_x' 'date' 'time' 'url'
 'fp1_date' 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time'
 'quali_date' 'quali_time' 'sprint_date' 'sprint_time' 'name_y']


<font size = "5">

Try it yourself!

- Rename the columns "name_x" and "name_y" <br>
in the dataset "races_merge_pitfall" to <br>
 "race_name" and "circuit_name"

$\quad$ HINT: Create a dictionary and use "pd.rename()"

In [26]:
# Write your own code
dict_rename_races_merge_pitfallx = {"name_x": "race_name"}
dict_rename_races_merge_pitfally = {"name_y": "circuit_name"}


dict_rename_races = {"name": "race_name"}
races_merge_pitfall_renamed1 = races_merge_pitfall.rename(columns = dict_rename_races_merge_pitfallx)
races_merge_pitfall_renamed2 = races_merge_pitfall_renamed1.rename(columns = dict_rename_races_merge_pitfally)

races_merge_pitfall_renamed2

,raceId,year,round,circuitId,race_name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,circuit_name
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Albert Park Grand Prix Circuit
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Sepang International Circuit
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Shanghai International Circuit
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Bahrain International Circuit
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,Circuit de Barcelona-Catalunya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1116,2023,19,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,Circuit of the Americas
1098,1117,2023,20,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,Autódromo Hermanos Rodríguez
1099,1118,2023,21,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,Autódromo José Carlos Pace
1100,1119,2023,22,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,2023-11-17,08:00:00,2023-11-18,04:30:00,2023-11-18,08:00:00,\N,\N,Las Vegas Strip Street Circuit


<font size = "5">

Try it yourself!

- Merge the column "alt", "lng", and "lat" into the races data <br>
using "pd.merge()

In [27]:
# Write your own code
races_df = pd.merge(races[["alt", "lng", "lat"]], on="race_id")

# The following code merges the raw data
# which has the "name" column in "races_raw" and "circuits_raw"

races_merge_pitfall = pd.merge(races_raw,
                               circuits_raw[["circuitId","name"]],
                               on = "circuitId",
                               how = "left")

# Python will internally rename the columns "name_x" (for the left dataset)
# and "name_y" (for the right dataset)


NameError: name 'races_df' is not defined

# <span style="color:darkblue"> III. Concat </span>


<font size = "5">

Use ".query()" to split data into different parts

In [15]:
circuits_spain = circuits.query('country == "Spain"')
circuits_usa   = circuits.query('country == "United States" | country == "USA"')
circuits_malaysia = circuits.query('country == "Malaysia"')

<font size = "5">

Cocatenate data back together

- Useful if there are datasets split by geography...
- year, or other subgroup

In [16]:
# Works best if columns are identical
# There are also other advanced options if they are not 
# https://pandas.pydata.org/docs/reference/api/pandas.concat.html

circuits_concat = pd.concat([circuits_spain,circuits_usa])


<font size = "5">

Try it yourself!

- Concatenate the USA and Malaysia datasets



In [28]:
# Write your own code

circuits_concat_us_mal = pd.concat([circuits_usa,circuits_malaysia])



